# Job creation

In [1]:
from azure.ai.ml import command, Input, MLClient, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute, Environment, Data, AzureBlobDatastore
from dotenv import load_dotenv
from azure.ai.ml.constants import AssetTypes, InputOutputModes

import os
from datetime import datetime

In [2]:
load_dotenv()

SUBSCRIPTION_ID = os.environ.get("SUBSCRIPTION_ID")
RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP")
WORKSPACE_NAME = os.environ.get("WORKSPACE_NAME")
DATASTORE_NAME = os.environ.get("DATASTORE_NAME")
ACCOUNT_NAME = os.environ.get("ACCOUNT_NAME")
CONTAINER_NAME = os.environ.get("CONTAINER_NAME")
COMPUTE_CLUSTER_NAME = os.environ.get("COMPUTE_NAME")
LOCATION = os.environ.get("LOCATION")
CONNECTION_KEY = os.environ.get("CONNECTION_KEY")
DATASET_NAME= os.environ.get("DATASET_NAME")

In [4]:
# connect to AML
ml_client = MLClient(
    DefaultAzureCredential(),
    SUBSCRIPTION_ID,
    RESOURCE_GROUP,
    WORKSPACE_NAME
)

In [5]:
# get prepared dataset (should be saved as data asset inside the workspace )
data_asset = ml_client.data.get(name="review-prepped", version="1")
data_asset.path

'azureml://subscriptions/8af766d1-2a08-48cd-8f6e-71c0ecdf262f/resourcegroups/prof-cs-rg/workspaces/worksp-proj-cs/datastores/dstoreprojcs/paths/UI/2024-04-03_132507_UTC/processed_reviews.csv'

In [7]:
# example to train a job in AML - it works
job = command(
    inputs=dict(
        prep_data=data_asset.path,
        registered_model_name="naive_bayes_baseline"
    ),
    code="src/train",
    command="python train.py --prep_data ${{inputs.prep_data}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="keras-env@latest",
    experiment_name="reviews_analysis__train",
    display_name="amazon_reviews__train",
    compute=COMPUTE_CLUSTER_NAME
)

returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor the job here : ", aml_url)

Monitor the job here :  https://ml.azure.com/runs/quiet_coat_fk0dcqvwjd?wsid=/subscriptions/8af766d1-2a08-48cd-8f6e-71c0ecdf262f/resourcegroups/prof-cs-rg/workspaces/worksp-proj-cs&tid=cf36141c-ddd7-45a7-b073-111f66d0b30c
